<a href="https://colab.research.google.com/github/JINDOT/Data_Analysis_/blob/main/Online_Retail_RFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Onlie Retail 를 이용한 RFM 분석

## 필요 라이브러리 및 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 전처리를 해야하기 때문에 raw 라고 일단 하자.
raw = pd.read_csv('/content/drive/MyDrive/csv/online_retail.csv')

In [ ]:
raw.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
raw.shape

(541909, 8)

## 데이터 전처리

### 유효 데이터 추출

In [ ]:
# 전 작업이랑 동일
# CustomerID 존재, Quantity, UnitPrice 가 0보다 큰 데이터만 사용
raw_valid = raw[(raw['CustomerID'].notnull()) & (raw['Quantity'] > 0) & (raw['UnitPrice'] > 0)]
raw_valid.shape

(397884, 8)

### 총 구매 금액

In [ ]:
# TotalPrice
raw_valid['TotalPrice'] = raw_valid['Quantity'] * raw_valid['UnitPrice']
raw_valid.head(1)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.3


In [ ]:
raw_valid.describe()

,Quantity,UnitPrice,CustomerID,TotalPrice
count,397884.000000,397884.000000,397884.000000,397884.000000
mean,12.988238,3.116488,15294.423453,22.397000
std,179.331775,22.097877,1713.141560,309.071041
min,1.000000,0.001000,12346.000000,0.001000
25%,2.000000,1.250000,13969.000000,4.680000
50%,6.000000,1.950000,15159.000000,11.800000
75%,12.000000,3.750000,16795.000000,19.800000
max,80995.000000,8142.750000,18287.000000,168469.600000


TotalPrice 를 자세히 보면 최대값이 평균값 보다 유독 큰 금액으로 나오는 것이 보인다.(= 이상치)

### 이상치 제거

In [ ]:
# 해당 주문건 확인
raw_valid[raw_valid['TotalPrice'] > 160000]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
540421,581483,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,2011-12-09 09:15:00,2.08,16446.0,United Kingdom,168469.6


총 주문 금액 160,000 이상인 주문건을 검색했을 때, 16446 고객의 주문건이 검색된다. Quantity(주문량)을 보았을 때 굉장히 많은 것을 볼 수 있다.

In [ ]:
# 해당 주문건 고객 확인
raw_valid[raw_valid['CustomerID'] == 16446]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
194354,553573,22980,PANTRY SCRUBBING BRUSH,1,2011-05-18 09:52:00,1.65,16446.0,United Kingdom,1.65
194355,553573,22982,PANTRY PASTRY BRUSH,1,2011-05-18 09:52:00,1.25,16446.0,United Kingdom,1.25
540421,581483,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,2011-12-09 09:15:00,2.08,16446.0,United Kingdom,168469.60


다른 품목 또한 비슷한 양으로 주문했다면 이상치가 아니겠지만 현재로서는 주문오류로 볼 수 있다.

In [ ]:
# 이상치 제거
raw_valid = raw_valid[raw_valid['TotalPrice'] < 160000].copy()
raw_valid.shape

(397883, 9)

이와 같이 실제 비지니스에서도 이상치를 무조건적으로 제거하기 보다는 여러 데이터를 확인해보고 제거해주어야 한다.

예를 들어 한 품목에 대해서 대량 주문 계약이었던 상황도 고려해보아야한다.

### 중복 데이터
> 중복 데이터는 여러 이유로 발생한다. 네트워크 통신 문제나 쇼핑몰 UI 구성 등 다양하다.

> 예를 들어 주문을 하고 새로고침 을 실행했을 때, 주문 트랜젝션이 다시 들어가는 이슈가 발생할 수 있다.

In [ ]:
# 중복 데이터 확인
# keep='first', 'last', False
raw_valid[raw_valid.duplicated(keep='first')].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
517,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,2010-12-01 11:45:00,1.25,17908.0,United Kingdom,1.25
527,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,2010-12-01 11:45:00,2.10,17908.0,United Kingdom,2.10
537,536409,22900,SET 2 TEA TOWELS I LOVE LONDON,1,2010-12-01 11:45:00,2.95,17908.0,United Kingdom,2.95
539,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,2010-12-01 11:45:00,4.95,17908.0,United Kingdom,4.95
555,536412,22327,ROUND SNACK BOXES SET OF 4 SKULLS,1,2010-12-01 11:49:00,2.95,17920.0,United Kingdom,2.95


In [ ]:
raw_valid[raw_valid.duplicated(keep='last')].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
485,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,2010-12-01 11:45:00,4.95,17908.0,United Kingdom,4.95
489,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,2010-12-01 11:45:00,2.10,17908.0,United Kingdom,2.10
494,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,2010-12-01 11:45:00,1.25,17908.0,United Kingdom,1.25
521,536409,22900,SET 2 TEA TOWELS I LOVE LONDON,1,2010-12-01 11:45:00,2.95,17908.0,United Kingdom,2.95
548,536412,22327,ROUND SNACK BOXES SET OF 4 SKULLS,1,2010-12-01 11:49:00,2.95,17920.0,United Kingdom,2.95


In [ ]:
raw_valid[raw_valid.duplicated(keep=False)].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
485,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,2010-12-01 11:45:00,4.95,17908.0,United Kingdom,4.95
489,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,2010-12-01 11:45:00,2.10,17908.0,United Kingdom,2.10
494,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,2010-12-01 11:45:00,1.25,17908.0,United Kingdom,1.25
517,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,2010-12-01 11:45:00,1.25,17908.0,United Kingdom,1.25
521,536409,22900,SET 2 TEA TOWELS I LOVE LONDON,1,2010-12-01 11:45:00,2.95,17908.0,United Kingdom,2.95


In [ ]:
# 중복 데이터 제거
df = raw_valid.drop_duplicates().copy()
df.shape

(392691, 9)

이 작업을 마지막으로 전처리가 끝났다.

## RFM

>***Rencency*** : 얼마나 최근에 구매했는가

>***Frequency*** : 얼마나 자주 구매했는가

>***Monetary*** : 얼마나 많은 금액을 지출했는가

즉, **가장 최근에 구매할수록, 자주 구매할수록, 많은 금액을 결제할수록 충성고객일 확률이 높다.**이렇게 고객들을 알고 그에 맞춰 충성고객일수록 더욱 많은 관심을 가질 수 있도록 그리고 아닌 고객일수록 좀 더 자회사 제품에 관심을 가질 수 있도록 만들 수 있다.


>***RFM 모형***
* Scoring 기법: RFM의 요인을 각각 5등급으로 등간격으로 분류하는 방법이다.
* 현재 개발된 RFM 모형은 크게 4가지로 분류 할 수 있다. 무조건적인 모델은 아니지만 참고할만 한 4가지 방법이다.
    * 모델1. RFM 각 요소의 20% rule의 적용
    * 모델2. 비율 척도에 의한 양적인 정도의 차이에 따른 등간격의 5등급 분류
    * 모델3. 상하 20%를 제외한 등간격 척도에 의한 그룹 분류
    * 모델4. 군집 분석에 의한 각 요소 별 5개의 그룹 분류
* Data Mining 기법을 이용한 모형
* 회귀분석
* 선형 회귀 분석을 이용한 모형: 고객의 구매 최근성, 구매 빈도, 구매 금액 등 고객의 수익 기여도를 나타내는 세가지 지표들의 선형결합으로 세가지 지표들을 점수화 한다.
* 다중 회귀 분석을 이용한 모형: 각 고객의 구매 행동을 나타내는 R,F,M의 변수들을 독립변수로하고 고객의 미래 구매 행동을 예측하는 기법.
* 신경망을 적용한 모형: 로지스틱 회귀 모형을 보완하는 차원에서 연구.
* 확률적 RFM모형:Colombo와 Weina의 확률적인 RFM모형은 과거의 고객의 응답 이력으로 고객의 미래 응답을 예측하는 행동모델이다.